In [1]:
import pandas as pd
#import sklearn as skr
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta # ?

In [4]:
# add tiemstamp to dataset
def add_timestamp(df, time_col='report_date'):
    # 目的是不改变原来的数据，其实没啥用，变量都重新赋值了，原来的数据都已经丢弃了
    data = df.copy()
    data['date'] = pd.to_datetime(data[time_col], format= "%Y%m%d")
    data['day'] = data['date'].dt.day
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year
    data['week'] = data['date'].dt.week
    data['weekday'] = data['date'].dt.weekday
    return data.reset_index(drop=True)

# total amount
def get_total_balance(data, date='2014-03-31'):
    df_tmp = data.copy()
    # 根据date特征，而不是某个具体的日期
    df_tmp = df_tmp.groupby(['date'])['total_purchase_amt','total_redeem_amt'].sum()
    # 变成df
    df_tmp.reset_index(inplace=True)
    # 过滤日期
    return df_tmp[(df_tmp['date']>= date)].reset_index(drop=True)

# Generate the test data
def generate_test_data(df):
    data = df.copy()
    start = datetime.datetime(2014,9,1)
    testdata = []
    # 自动生成2014.9.1到2014.10.15之间的数据，第二列和第三列暂时空着
    while start != datetime.datetime(2014,10,15):
        temp = [start, np.nan, np.nan]
        testdata.append(temp)
        start += datetime.timedelta(days=1)
    testdata = pd.DataFrame(testdata)
    testdata.columns = data.columns
    data = pd.concat([data, testdata], axis=0)
    return data.reset_index(drop=True)

# Load user's information
def load_user_information(path='user_profile_table.csv'):
    return pd.read_csv(path)

In [5]:
# 载入数据
balance_data = pd.read_csv('data/user_balance_table.csv')
balance_data = add_timestamp(balance_data)

total_balance = get_total_balance(balance_data, date='2014-03-01')
total_balance = generate_test_data(total_balance)
total_balance = add_timestamp(total_balance, 'date')

D:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [12]:
# 创建数据的深层拷贝

total = total_balance.copy()

In [14]:
total.head()

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday
0,2014-03-01,362865580.0,211279011.0,1,3,2014,9,5
1,2014-03-02,276202230.0,246199417.0,2,3,2014,9,6
2,2014-03-03,505305862.0,513017360.0,3,3,2014,10,0
3,2014-03-04,524146340.0,250562978.0,4,3,2014,10,1
4,2014-03-05,454295491.0,209072753.0,5,3,2014,10,2


In [82]:
data = total.copy()

In [51]:
month_index = 4
data = data[['date','total_purchase_amt', 'total_redeem_amt', 'weekday', 'day', 'week', 'month']]
# 从 4月1日 到 9月1日
data = data[(data['date'] >= datetime.datetime(2014,3,1)) & (data['date'] < datetime.datetime(2014, month_index, 1))]

# 统计翌日因子
mean_of_each_weekday = data[['weekday', 'total_purchase_amt','total_redeem_amt']].groupby('weekday', as_index=False).mean()
# 给列名加后缀
for col in ['total_purchase_amt','total_redeem_amt']:    
    mean_of_each_weekday = mean_of_each_weekday.rename(columns={col: col + '_weekdaymean'})
mean_of_each_weekday['total_purchase_amt_weekdaymean'] /= np.mean(data['total_purchase_amt'])
mean_of_each_weekday['total_redeem_amt_weekdaymean'] /= np.mean(data['total_redeem_amt'])

In [52]:
mean_of_each_weekday

,weekday,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,0,1.227407,1.342971
1,1,1.273349,1.161819
2,2,1.093144,1.189935
3,3,1.149968,0.992409
4,4,0.899730,1.002284
5,5,0.744373,0.561287
6,6,0.695267,0.818583


In [53]:
# 合并统计结果到原数据集，根据这一个数值进行合并会有很多重复！！！
data = pd.merge(data, mean_of_each_weekday, on='weekday', how='left')
data.head()

,date,total_purchase_amt,total_redeem_amt,weekday,day,week,month,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,2014-03-01,362865580.0,211279011.0,5,1,9,3,0.744373,0.561287
1,2014-03-02,276202230.0,246199417.0,6,2,9,3,0.695267,0.818583
2,2014-03-03,505305862.0,513017360.0,0,3,10,3,1.227407,1.342971
3,2014-03-04,524146340.0,250562978.0,1,4,10,3,1.273349,1.161819
4,2014-03-05,454295491.0,209072753.0,2,5,10,3,1.093144,1.189935


In [54]:
# 分别统计翌日在(1~31)号出现的频次(最后统计的是date的次数？？？)
weekday_count = data[['day','weekday','date']].groupby(['day','weekday'],as_index=False).count()
# 每个月的全部31个应该都是1
weekday_count.head()

,day,weekday,date
0,1,5,1
1,2,6,1
2,3,0,1
3,4,1,1
4,5,2,1


In [55]:
weekday_count = pd.merge(weekday_count, mean_of_each_weekday, on='weekday')
weekday_count.head()

,day,weekday,date,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,1,5,1,0.744373,0.561287
1,8,5,1,0.744373,0.561287
2,15,5,1,0.744373,0.561287
3,22,5,1,0.744373,0.561287
4,29,5,1,0.744373,0.561287


In [56]:
# 依据频次对翌日因子进行加权，获得日期因子（但实际上权重都是1）
weekday_count['total_purchase_amt_weekdaymean'] *= weekday_count['date'] / len(np.unique(data['month']))
weekday_count['total_redeem_amt_weekdaymean'] *= weekday_count['date'] / len(np.unique(data['month']))
day_rate = weekday_count.drop(['weekday','date'], axis=1).groupby('day', as_index=False).sum()

In [61]:
day_rate.head()

,day,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,1,0.744373,0.561287
1,2,0.695267,0.818583
2,3,1.227407,1.342971
3,4,1.273349,1.161819
4,5,1.093144,1.189935


In [71]:
# 将训练集中所有日期的均值剔除日期残差得到 base
day_mean = data[['day', 'total_purchase_amt','total_redeem_amt']].groupby('day',as_index=False).mean()
day_pre = pd.merge(day_mean, day_rate, on='day', how='left')

In [72]:
day_pre.head(10)

,day,total_purchase_amt,total_redeem_amt,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,1,362865580.0,211279011.0,0.744373,0.561287
1,2,276202230.0,246199417.0,0.695267,0.818583
2,3,505305862.0,513017360.0,1.227407,1.342971
3,4,524146340.0,250562978.0,1.273349,1.161819
4,5,454295491.0,209072753.0,1.093144,1.189935
5,6,561787770.0,243149884.0,1.149968,0.992409
6,7,380139779.0,291087220.0,0.899730,1.002284
7,8,243274169.0,140323202.0,0.744373,0.561287
8,9,244752519.0,206312503.0,0.695267,0.818583
9,10,497338076.0,308040624.0,1.227407,1.342971


In [73]:
day_pre['total_purchase_amt'] /= day_pre['total_purchase_amt_weekdaymean']
day_pre['total_redeem_amt'] /= day_pre['total_redeem_amt_weekdaymean']

In [74]:
day_pre.head(10)

,day,total_purchase_amt,total_redeem_amt,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,1,4.874780e+08,3.764186e+08,0.744373,0.561287
1,2,3.972607e+08,3.007629e+08,0.695267,0.818583
2,3,4.116858e+08,3.820017e+08,1.227407,1.342971
3,4,4.116282e+08,2.156643e+08,1.273349,1.161819
4,5,4.155863e+08,1.757009e+08,1.093144,1.189935
5,6,4.885245e+08,2.450098e+08,1.149968,0.992409
6,7,4.225041e+08,2.904239e+08,0.899730,1.002284
7,8,3.268175e+08,2.500024e+08,0.744373,0.561287
8,9,3.520267e+08,2.520361e+08,0.695267,0.818583
9,10,4.051943e+08,2.293724e+08,1.227407,1.342971


In [75]:
# 迭代每行数据：generator object，可以list化后查看
for index, row in day_pre.iterrows():
    print(index)
    print(row)
    print('\n')

0
day                               1.000000e+00
total_purchase_amt                4.874780e+08
total_redeem_amt                  3.764186e+08
total_purchase_amt_weekdaymean    7.443733e-01
total_redeem_amt_weekdaymean      5.612874e-01
Name: 0, dtype: float64


1
day                               2.000000e+00
total_purchase_amt                3.972607e+08
total_redeem_amt                  3.007629e+08
total_purchase_amt_weekdaymean    6.952670e-01
total_redeem_amt_weekdaymean      8.185830e-01
Name: 1, dtype: float64


2
day                               3.000000e+00
total_purchase_amt                4.116858e+08
total_redeem_amt                  3.820017e+08
total_purchase_amt_weekdaymean    1.227407e+00
total_redeem_amt_weekdaymean      1.342971e+00
Name: 2, dtype: float64


3
day                               4.000000e+00
total_purchase_amt                4.116282e+08
total_redeem_amt                  2.156643e+08
total_purchase_amt_weekdaymean    1.273349e+00
total_redeem_amt_week

In [76]:
# 生成测试集数据
for index, row in day_pre.iterrows():
    # 难道不是不包括31号，然后自动停止？？？？？？？？
    if month_index in (2,4,6,9) and row['day'] == 31:
        break
    # 没有date特征？？
    day_pre.loc[index, 'date'] = datetime.datetime(2014, month_index, int(row['day']))

In [77]:
day_pre.head(10)

,day,total_purchase_amt,total_redeem_amt,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean,date
0,1,4.874780e+08,3.764186e+08,0.744373,0.561287,2014-04-01
1,2,3.972607e+08,3.007629e+08,0.695267,0.818583,2014-04-02
2,3,4.116858e+08,3.820017e+08,1.227407,1.342971,2014-04-03
3,4,4.116282e+08,2.156643e+08,1.273349,1.161819,2014-04-04
4,5,4.155863e+08,1.757009e+08,1.093144,1.189935,2014-04-05
5,6,4.885245e+08,2.450098e+08,1.149968,0.992409,2014-04-06
6,7,4.225041e+08,2.904239e+08,0.899730,1.002284,2014-04-07
7,8,3.268175e+08,2.500024e+08,0.744373,0.561287,2014-04-08
8,9,3.520267e+08,2.520361e+08,0.695267,0.818583,2014-04-09
9,10,4.051943e+08,2.293724e+08,1.227407,1.342971,2014-04-10


In [81]:
mean_of_each_weekday

,weekday,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,0,1.227407,1.342971
1,1,1.273349,1.161819
2,2,1.093144,1.189935
3,3,1.149968,0.992409
4,4,0.899730,1.002284
5,5,0.744373,0.561287
6,6,0.695267,0.818583


In [69]:
# 基于base与翌日因子获得最后的预测结果
day_pre['weekday'] = day_pre.date.dt.weekday
day_pre = day_pre[['date','weekday', 'total_purchase_amt','total_redeem_amt']]
# 合并后，相同的特征使用的是
day_pre = pd.merge(day_pre, mean_of_each_weekday, on='weekday')

In [70]:
day_pre.head()

,date,weekday,total_purchase_amt,total_redeem_amt,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,2014-04-01,1.0,4.874780e+08,3.764186e+08,1.273349,1.161819
1,2014-04-08,1.0,3.268175e+08,2.500024e+08,1.273349,1.161819
2,2014-04-15,1.0,3.861060e+08,4.325752e+08,1.273349,1.161819
3,2014-04-22,1.0,2.575323e+08,2.459335e+08,1.273349,1.161819
4,2014-04-29,1.0,2.152831e+08,2.761616e+08,1.273349,1.161819


In [46]:
day_pre['total_purchase_amt'] *= day_pre['total_purchase_amt_weekdaymean']
day_pre['total_redeem_amt'] *= day_pre['total_redeem_amt_weekdaymean']

In [47]:
day_pre.head()

,date,weekday,total_purchase_amt,total_redeem_amt,total_purchase_amt_weekdaymean,total_redeem_amt_weekdaymean
0,2014-04-01,1.0,6.207296e+08,4.373304e+08,1.273349,1.161819
1,2014-04-08,1.0,4.161527e+08,2.904576e+08,1.273349,1.161819
2,2014-04-15,1.0,4.916477e+08,5.025743e+08,1.273349,1.161819
3,2014-04-22,1.0,3.279285e+08,2.857304e+08,1.273349,1.161819
4,2014-04-29,1.0,2.741305e+08,3.208499e+08,1.273349,1.161819


In [48]:
day_pre = day_pre.sort_values('date')[['date', 'total_purchase_amt','total_redeem_amt']]

In [49]:
day_pre.head()

,date,total_purchase_amt,total_redeem_amt
0,2014-04-01,6.207296e+08,4.373304e+08
5,2014-04-02,4.342629e+08,3.578884e+08
10,2014-04-03,4.734257e+08,3.791019e+08
14,2014-04-04,3.703544e+08,2.161569e+08
18,2014-04-05,3.093513e+08,9.861873e+07


In [83]:
# 定义根据时间序列规则生成预测结果的方法

def generate_base(df, month_index):
    # 选中固定时间段的数据集
    data = df.copy()
    data = data[['date','total_purchase_amt', 'total_redeem_amt', 'weekday', 'day', 'week', 'month']]
    # 从 4月1日 到 9月1日
    data = data[(data['date'] >= datetime.datetime(2014,3,1)) & (data['date'] < datetime.datetime(2014, month_index, 1))]
    
    # 统计翌日因子
    mean_of_each_weekday = data[['weekday', 'total_purchase_amt','total_redeem_amt']].groupby('weekday', as_index=False).mean()
    for col in ['total_purchase_amt','total_redeem_amt']:
        # 给列名加后缀
        mean_of_each_weekday = mean_of_each_weekday.rename(columns={col: col + '_weekdaymean'})
    mean_of_each_weekday['total_purchase_amt_weekdaymean'] /= np.mean(data['total_purchase_amt'])
    mean_of_each_weekday['total_redeem_amt_weekdaymean'] /= np.mean(data['total_redeem_amt'])

    # 合并统计结果到原数据集，根据这一个数值进行合并会有很多重复！！！
    data = pd.merge(data, mean_of_each_weekday, on='weekday', how='left')

    # 分别统计翌日在(1~31)号出现的频次
    weekday_count = data[['day','weekday','date']].groupby(['day','weekday'],as_index=False).count()
    weekday_count = pd.merge(weekday_count, mean_of_each_weekday, on='weekday')

    # 依据频次对翌日因子进行加权，获得日期因子
    weekday_count['total_purchase_amt_weekdaymean'] *= weekday_count['date'] / len(np.unique(data['month']))
    weekday_count['total_redeem_amt_weekdaymean'] *= weekday_count['date'] / len(np.unique(data['month']))
    day_rate = weekday_count.drop(['weekday','date'],axis=1).groupby('day', as_index=False).sum()

    # 将训练集中所有日期的（交易量）均值剔除日期残差得到base
    day_mean = data[['day', 'total_purchase_amt','total_redeem_amt']].groupby('day',as_index=False).mean()
    day_pre = pd.merge(day_mean, day_rate, on='day', how='left')
    day_pre['total_purchase_amt'] /= day_pre['total_purchase_amt_weekdaymean']
    day_pre['total_redeem_amt'] /= day_pre['total_redeem_amt_weekdaymean']

    # 生成测试集数据
    for index, row in day_pre.iterrows():
        if month_index in (2,4,6,9) and row['day'] == 31:
            break
        # 生成新的date特征,那前面为何要删除，是因为groupby时不方便吗？？
        day_pre.loc[index, 'date'] = datetime.datetime(2014, month_index, int(row['day']))

    # 基于base与翌日因子获得最后的预测结果
    day_pre['weekday'] = day_pre.date.dt.weekday
    day_pre = day_pre[['date','weekday']+['total_purchase_amt','total_redeem_amt']]
    day_pre = pd.merge(day_pre, mean_of_each_weekday,on='weekday')
    day_pre['total_purchase_amt'] *= day_pre['total_purchase_amt_weekdaymean']
    day_pre['total_redeem_amt'] *= day_pre['total_redeem_amt_weekdaymean']

    day_pre = day_pre.sort_values('date')[['date']+['total_purchase_amt','total_redeem_amt']]
    return day_pre

In [84]:
# 生成预测结果（以及残差）

base_list = []
for i in range(4, 10):
    base_list.append(generate_base(data, i).reset_index(drop=True))

# 直接把列表里所有的df都拼接起来吗？？？
base = pd.concat(base_list).reset_index(drop=True)
for col in ['total_purchase_amt','total_redeem_amt']:
    base = base.rename(columns={col: col+'_base'})

data = pd.merge(data.reset_index(drop=True), 
                base.reset_index(drop=True), on='date', how='left').reset_index(drop=True)

# 为何除一下是residual
data['purchase_residual'] = data['total_purchase_amt'] / data['total_purchase_amt_base']
data['redeem_residual'] = data['total_redeem_amt'] / data['total_redeem_amt_base']

In [91]:
# 为何base这里有2014-09的数据，原始数据里都没有，难道是前面根据weekday合并的时候填充的？？？
base.head().append(base.tail())

,date,total_purchase_amt_base,total_redeem_amt_base
0,2014-04-01,6.207296e+08,4.373304e+08
1,2014-04-02,4.342629e+08,3.578884e+08
2,2014-04-03,4.734257e+08,3.791019e+08
3,2014-04-04,3.703544e+08,2.161569e+08
4,2014-04-05,3.093513e+08,9.861873e+07
178,2014-09-26,2.217015e+08,3.063300e+08
179,2014-09-27,1.736759e+08,2.181428e+08
180,2014-09-28,1.809855e+08,2.492289e+08
181,2014-09-29,2.953817e+08,3.515598e+08
182,2014-09-30,2.975587e+08,3.454322e+08


In [85]:
data.head()

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday,total_purchase_amt_base,total_redeem_amt_base,purchase_residual,redeem_residual
0,2014-03-01,362865580.0,211279011.0,1,3,2014,9,5,NaN,NaN,NaN,NaN
1,2014-03-02,276202230.0,246199417.0,2,3,2014,9,6,NaN,NaN,NaN,NaN
2,2014-03-03,505305862.0,513017360.0,3,3,2014,10,0,NaN,NaN,NaN,NaN
3,2014-03-04,524146340.0,250562978.0,4,3,2014,10,1,NaN,NaN,NaN,NaN
4,2014-03-05,454295491.0,209072753.0,5,3,2014,10,2,NaN,NaN,NaN,NaN


balance数据只有2014-04-01到2014-08-31的，所以其他的都是空值

In [93]:
data.sample(10)

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday,total_purchase_amt_base,total_redeem_amt_base,purchase_residual,redeem_residual
42,2014-04-12,177642053.0,123295320.0,12,4,2014,15,5,2.567225e+08,1.964572e+08,0.691961,0.627594
139,2014-07-18,208959595.0,208671287.0,18,7,2014,29,4,2.602192e+08,2.904045e+08,0.803014,0.718554
80,2014-05-20,453955303.0,260040720.0,20,5,2014,21,1,3.824718e+08,3.122205e+08,1.186899,0.832875
67,2014-05-07,417327518.0,239372999.0,7,5,2014,19,2,3.254117e+08,2.321467e+08,1.282460,1.031128
188,2014-09-05,NaN,NaN,5,9,2014,36,4,2.909838e+08,2.508833e+08,NaN,NaN
93,2014-06-02,158219402.0,170409506.0,2,6,2014,23,0,3.269797e+08,2.772971e+08,0.483881,0.614538
152,2014-07-31,191728916.0,277194379.0,31,7,2014,31,3,7.029328e+08,6.113257e+08,0.272756,0.453432
123,2014-07-02,384555819.0,328950951.0,2,7,2014,27,2,2.725488e+08,2.272268e+08,1.410961,1.447677
114,2014-06-23,232227670.0,373779624.0,23,6,2014,26,0,3.156586e+08,3.006415e+08,0.735693,1.243274
133,2014-07-12,177644343.0,149081488.0,12,7,2014,28,5,2.048239e+08,1.648825e+08,0.867303,0.904168


In [108]:
balance_data[balance_data['report_date']==20140831]

,user_id,report_date,tBalance,yBalance,total_purchase_amt,direct_purchase_amt,purchase_bal_amt,purchase_bank_amt,total_redeem_amt,consume_amt,...,category1,category2,category3,category4,date,day,month,year,week,weekday
37,1,20140831,20437,20435,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2014-08-31,31,8,2014,35,6
221,6,20140831,1236,1236,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2014-08-31,31,8,2014,35,6
337,8,20140831,171,0,2971,2971,2971,0,2800,2800,...,0.0,0.0,0.0,2800.0,2014-08-31,31,8,2014,35,6
400,11,20140831,1071728,1101956,123,0,0,0,30351,0,...,NaN,NaN,NaN,NaN,2014-08-31,31,8,2014,35,6
579,23,20140831,179551,41436,138115,138111,138111,0,0,0,...,NaN,NaN,NaN,NaN,2014-08-31,31,8,2014,35,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839355,28009,20140831,554,554,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2014-08-31,31,8,2014,35,6
2839471,28013,20140831,1903,1903,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2014-08-31,31,8,2014,35,6
2839663,28017,20140831,74947,75539,8,0,0,0,600,600,...,0.0,0.0,0.0,600.0,2014-08-31,31,8,2014,35,6
2840144,28019,20140831,15201,15200,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2014-08-31,31,8,2014,35,6


In [109]:
# 对结果表重命名

data = data[['date','purchase_residual','redeem_residual','total_purchase_amt_base', 'total_redeem_amt_base']]
# 可以用df.rename方法
for col in data.columns:
    if col == 'date':
        data[col] = data[col].astype(str)
        data[col] = data[col].str.replace('-', '')
data.columns = [['date'] + ['total_purchase_amt','total_redeem_amt'] + ['total_purchase_predicted_by_cycle',
                                                                        'total_redeem_predicted_by_cycle'] ]

In [110]:
# 保存预测结果到本地

data.to_csv('data/base.csv',index=False)